In [21]:
pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.242.0
    Uninstalling sagemaker-2.242.0:
      Successfully uninstalled sagemaker-2.242.0
Note: you may need to restart the kernel to use updated packages.


In [122]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [21]:
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import os

In [22]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [23]:
local_source_code_dir = './'

In [24]:
entry_point_script2 = 's3://sagemaker-studio-879381274336-3q1czh5a9dt/stock-data-for-portfolio-optimization/finance_genetic_algorithm.tar.gz'

In [25]:
entry_point_script = "main.py"

In [26]:
input_data_s3_path = f"s3://sagemaker-studio-879381274336-3q1czh5a9dt/stock-data-for-portfolio-optimization/stock_daily_returns.csv"
output_data_s3_path = f"s3://sagemaker-studio-879381274336-3q1czh5a9dt/optimized_portfolios/"

In [27]:
print(sagemaker_session.boto_region_name)

us-east-2


In [28]:
processor = ScriptProcessor(
    image_uri="879381274336.dkr.ecr.us-east-2.amazonaws.com/python3-sagemaker-2:1.0-x86",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    command=['python3']
)

In [29]:
processor.run(
    code=entry_point_script,
    inputs=[
        ProcessingInput(
            source=input_data_s3_path,
            destination='/opt/ml/processing/input/data',
            s3_data_type='S3Prefix',
            s3_input_mode='File'
        )
    ],
    outputs=[
        ProcessingOutput(
            source='/opt/ml/processing/output/results',
            destination=output_data_s3_path
        )
    ],
    arguments=[
        '--input_data_path', os.path.join('/opt/ml/processing/input/data', 'stock_daily_returns.csv'),
        '--output_data_path', '/opt/ml/processing/output/results',
        '--num_generations', '200',
        '--population_size', '100',
        '--risk_free_rate', '0.025',
        '--crossover_rate', '0.7',
        '--mutation_rate', '0.15',
        '--elitism_count', '5'
    ]
)

INFO:sagemaker:Creating processing-job with name python3-sagemaker-2-2025-05-23-17-03-47-883


.............Successfully loaded daily returns data from: /opt/ml/processing/input/data/stock_daily_returns.csv



In [30]:
print(f"SageMaker Processing Job launched. Check CloudWatch logs for progress.")
print(f"Results will be in S3: {output_data_s3_path}")

SageMaker Processing Job launched. Check CloudWatch logs for progress.
Results will be in S3: s3://sagemaker-studio-879381274336-3q1czh5a9dt/optimized_portfolios/
